In [1]:
# from utils.compression import compress_directories, decompress_directories

# for base_path, batch_type in data_partition.items():
#     decompress_directories(
#         source_path=f'downloaded_data/{base_path}',
#         destination_path=f'data/{base_path}',
#         checksum_file_name="checksums.txt",
#         delete_compressed_files=True,
#         num_workers=26
#     )

In [3]:
# del session.logger
# del session

from pickle import NONE
from train.training_session_v0 import TrainingSessionV0
from config import TrainingConfigV0
import multiprocessing
from config import SimpleConvConfig
from models.simpleconv import SimpleConv
import torch

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=128,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=False,
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV0(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Pre-processing parameters
    # Brain
    new_freq=100,
    frequency_bands={"all": (0.5, 40)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.0,
    notch_filter=True,
    scaling="standard",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=256,
    use_clip_loss=True,
    use_mse_loss=True,
    alpha=0.6,
    random_test_size=10,
    seed=42,
)

session = TrainingSessionV0(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase1/objectives/CLIP_MSE",
    clear_cache=False,
    cache_name="cache/1",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2) // 2,
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
SimpleConv initialized with 8448160 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 7973376


Training Epoch 1: 100%|██████████| 135/135 [05:06<00:00,  2.27s/it]


Epoch 1 completed. Loss: 3.3592, Clip Loss: 5.0375, MSE Loss: 0.8417
Accuracy: 0.0499, Top 5: 0.1583, Top 10: 0.2434
Test unseen_subject completed. Accuracy: 0.0705, Top 5: 0.2010, Top 10: 0.3080
Test unseen_task completed. Accuracy: 0.0576, Top 5: 0.1725, Top 10: 0.2664
Test unseen_both completed. Accuracy: 0.0712, Top 5: 0.1737, Top 10: 0.2587
Testing completed in 0.46m.
Epoch 1 completed in 5.56m. 0.04m per recording.


Training Epoch 2: 100%|██████████| 135/135 [05:15<00:00,  2.34s/it]


Epoch 2 completed. Loss: 2.7826, Clip Loss: 4.3002, MSE Loss: 0.5063
Accuracy: 0.0900, Top 5: 0.2550, Top 10: 0.3697
Test unseen_subject completed. Accuracy: 0.1105, Top 5: 0.2735, Top 10: 0.3825
Test unseen_task completed. Accuracy: 0.1018, Top 5: 0.2851, Top 10: 0.4047
Test unseen_both completed. Accuracy: 0.0963, Top 5: 0.2675, Top 10: 0.4113
Testing completed in 0.46m.
Epoch 2 completed in 5.72m. 0.04m per recording.


...

Epoch 25 completed. Loss: 0.6780, Clip Loss: 0.8160, MSE Loss: 0.4710
Accuracy: 0.7604, Top 5: 0.9351, Top 10: 0.9683
Test unseen_subject completed. Accuracy: 0.1575, Top 5: 0.3615, Top 10: 0.4505
Test unseen_task completed. Accuracy: 0.1567, Top 5: 0.3714, Top 10: 0.4953
Test unseen_both completed. Accuracy: 0.1263, Top 5: 0.2750, Top 10: 0.3925
Testing completed in 0.42m.
Epoch 25 completed in 6.40m. 0.05m per recording.


Training Epoch 26: 100%|██████████| 135/135 [05:55<00:00,  2.64s/it]


Epoch 26 completed. Loss: 1.7646, Clip Loss: 2.5176, MSE Loss: 0.6351
Accuracy: 0.4729, Top 5: 0.6632, Top 10: 0.7338
Test unseen_subject completed. Accuracy: 0.0325, Top 5: 0.1435, Top 10: 0.2435
Test unseen_task completed. Accuracy: 0.0369, Top 5: 0.1478, Top 10: 0.2407
Test unseen_both completed. Accuracy: 0.0275, Top 5: 0.1400, Top 10: 0.2150
Testing completed in 0.43m.
Epoch 26 completed in 6.37m. 0.05m per recording.


Training Epoch 27: 100%|██████████| 135/135 [04:19<00:00,  1.93s/it]


Epoch 27 completed. Loss: 2.9453, Clip Loss: 4.4905, MSE Loss: 0.6276
Accuracy: 0.1074, Top 5: 0.2814, Top 10: 0.3909
Test unseen_subject completed. Accuracy: 0.0510, Top 5: 0.1700, Top 10: 0.2600
Test unseen_task completed. Accuracy: 0.0560, Top 5: 0.1739, Top 10: 0.2630
Test unseen_both completed. Accuracy: 0.0488, Top 5: 0.1475, Top 10: 0.2275
Testing completed in 0.36m.
Epoch 27 completed in 4.69m. 0.03m per recording.
Early stopping at epoch 27. Highest top 10 accuracy at epoch 16.
Training completed.
unseen_subject: Acc: 0.2050, Top 5: 0.4280, Top 10: 0.5315
unseen_task: Acc: 0.2087, Top 5: 0.4420, Top 10: 0.5586
unseen_both: Acc: 0.1888, Top 5: 0.3950, Top 10: 0.4875


## CLIP MSE

In [4]:
# # del session.logger
# # del session

# from pickle import NONE
# from train.training_session_v0 import TrainingSessionV0
# from config import TrainingConfigV0
# import multiprocessing
# from config import SimpleConvConfig
# from models.simpleconv import SimpleConv
# import torch

# data_partition = {
#     "gwilliams2023": {
#         "testing_subjects": [19, 20, 21],
#         "testing_tasks": [0],
#     },
#     # "armeini2022": {
#     #     "testing_subjects": [],
#     #     "testing_tasks": [8, 9],
#     # },
# }

# model_config = SimpleConvConfig(
#     # Str to list of possible conditions
#     mel_normalization=False,
#     conditions={
#         "study": [],
#         "subject": [],
#     },
#     # Channels
#     in_channels=208,
#     out_channels=128,
#     hidden_dim=384,
#     dropout=0.2,
#     initial_batch_norm=True,
#     # Sensor layout settings
#     layout_dim=2,
#     layout_proj=True,
#     layout_scaling="minmax",
#     # Merger with spatial attn
#     merger=False,
#     merger_emb_type=None,
#     merger_emb_dim=0,
#     merger_channels=0,
#     merger_dropout=False,
#     merger_conditional=None,
#     # Inital
#     initial_linear=384,
#     initial_depth=1,
#     # Conditional layers
#     conditional_layers=False,
#     conditional_layers_dim=None,  # input or hidden_dim
#     # Conv layer overall structure
#     depth=6,
#     kernel_size=3,
#     growth=1.0,
#     dilation_growth=2,
#     dilation_period=5,
#     glu=1,
#     conv_dropout=0.2,
#     dropout_input=0.2,
#     batch_norm=True,
#     half=False,
#     # Quantizer
#     quantizer=False,
#     num_codebooks=0,
#     codebook_size=0,
#     quantizer_commitment=0,
#     quantizer_temp_init=0,
#     quantizer_temp_min=0,
#     quantizer_temp_decay=0,
#     # Transformers Encoders
#     transformer_input=None,
#     transformer_encoder_emb=None,
#     transformer_encoder_layers=0,
#     transformer_encoder_heads=0,
#     # Transformer Decoders
#     transformer_decoder_emb=None,
#     transformer_decoder_layers=0,
#     transformer_decoder_heads=0,
#     transformer_decoder_dim=0,
# )

# config = TrainingConfigV0(
#     brain_encoder_config=model_config,
#     data_partition=data_partition,
#     # Pre-processing parameters
#     # Brain
#     new_freq=100,
#     frequency_bands={"all": (0.5, 40)},
#     max_random_shift=1.0,
#     window_size=4,
#     window_stride=1,
#     brain_clipping=None,
#     baseline_window=0.0,
#     notch_filter=True,
#     scaling="standard",
#     delay=0.15,
#     # Hyperparameters
#     learning_rate=3e-4,
#     weight_decay=1e-4,
#     epochs=50,
#     batch_size=256,
#     use_clip_loss=True,
#     use_mse_loss=True,
#     alpha=0.6,
#     random_test_size=10,
#     seed=42,
# )

# config.notch_filter = False

# session = TrainingSessionV0(
#     config=config,
#     studies={study: "audio" for study in data_partition.keys()},
#     data_path="data",
#     save_path="saves/phase1/ablation/notch_filter/False",
#     clear_cache=True,
#     cache_name="cache/1",
# )

# try:
#     session.train(
#         device="cuda",
#         buffer_size=30,
#         num_workers=(multiprocessing.cpu_count() - 2) // 2,
#         max_cache_size=400,
#         current_epoch=0,
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

## No notch